In [2]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from dataset_wrapper import get_pet_datasets

In [11]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module): # This class inherits from nn.Module and defines the architecture of the neural network.
    #In PyTorch, nn.Module is the base class for all neural network modules. It provides a set of functionalities that are 
    # commonly required when building neural networks, such as automatic gradient computation, model saving/loading, and simple layer management.
    def __init__(self, num_classes=4):
        # In PyTorch, custom neural networks are created by subclassing the nn.Module class and defining layers inside its __init__() method. 
        #The layers you define as class members (e.g., self.conv1, self.fc1) are automatically registered and tracked by PyTorch.
        super(ConvNet, self).__init__()

        #super() is used to call the __init__ method of the parent class nn.Module, without super(), you would lose the setup required for PyTorch
        #to properly manage the layers, parameters, and gradients in your neural network model.
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(32*32*32, num_classes)
        
    def forward(self, x):
        #The forward() method defines how the input data flows through the network (i.e., the computation graph). 
        #It's where you define the sequence of operations (e.g., layers, activations) the input goes through to produce the output.
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
#hyperparameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

#getting datasets
train_dataset, val_dataset, test_dataset = get_pet_datasets(img_width=128, img_height=128,root_path='./data' )

#selecting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#our dataloader for training dataset
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [1/58], Loss: 2.2918
Epoch [1/5], Step [2/58], Loss: 2.9855
Epoch [1/5], Step [3/58], Loss: 11.4101
Epoch [1/5], Step [4/58], Loss: 12.8035
Epoch [1/5], Step [5/58], Loss: 13.4466
Epoch [1/5], Step [6/58], Loss: 11.1134
Epoch [1/5], Step [7/58], Loss: 10.7866
Epoch [1/5], Step [8/58], Loss: 9.7071
Epoch [1/5], Step [9/58], Loss: 2.9642
Epoch [1/5], Step [10/58], Loss: 6.0615
Epoch [1/5], Step [11/58], Loss: 5.1869
Epoch [1/5], Step [12/58], Loss: 6.1744
Epoch [1/5], Step [13/58], Loss: 4.9648
Epoch [1/5], Step [14/58], Loss: 3.7653
Epoch [1/5], Step [15/58], Loss: 2.5995
Epoch [1/5], Step [16/58], Loss: 4.8229
Epoch [1/5], Step [17/58], Loss: 4.7022
Epoch [1/5], Step [18/58], Loss: 2.7527
Epoch [1/5], Step [19/58], Loss: 3.2517
Epoch [1/5], Step [20/58], Loss: 3.5825
Epoch [1/5], Step [21/58], Loss: 3.4756
Epoch [1/5], Step [22/58], Loss: 2.4251
Epoch [1/5], Step [23/58], Loss: 1.7666
Epoch [1/5], Step [24/58], Loss: 3.0760
Epoch [1/5], Step [25/58], Loss: 2.9883
Epoc

In [15]:
#saves model to a dictionary on disk
def save_model():
    torch.save(model.state_dict(), 'model.ckpt')

save_model()

In [14]:
def test_model():
    model.eval()

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)
    # Disable gradient calculation for efficiency
    with torch.no_grad():
        correct = 0
        total = 0
        
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
    
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        print('Test Accuracy of the model on the 10000 test images: {:.2f} %'.format(100 * correct / total))

test_model()

Test Accuracy of the model on the 10000 test images: 40.08 %
